# CASE STUDY - topic modeling and feature engineering


[Feature engineering](https://en.wikipedia.org/wiki/Feature_engineering) is the process of using domain knowledge of your data to create features that can be leveraged by machine learning.  That is not a hard definition, because sometimes it is used in a context where features are transformed for machine learning, but the inclusion of domain knowledge is not implied.  

It is unfortunately common that for large datasets engineered features are not easy to create.  When there are many features generally only a small number play an important roll when it comes to prediction.  Furthermore,  domain insight is even more difficult to fold into the model when there are hundreds or thousands of features to keep in mind.  However, there is a middle ground---much of the worlds knowledge is locked up in language.  In this case study we will use topic modeling to gather insight from text.  Ideally, the result of these types of experiments would be shared with domain experts to further engineer features that are relevant when it comes to your business opportunity.

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from string import punctuation, printable
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

try:
    import pyLDAvis
    import pyLDAvis.sklearn
except:
    raise Exception("'pip install pyldavis' before running this notebook")

pyLDAvis.enable_notebook()    
plt.style.use('seaborn')
%matplotlib inline

## supress all warnings (not to be used during development)
import warnings
warnings.filterwarnings("ignore")

C:\Source-Files\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


## Synopsis

   >Goal:  AAVAIL has recently enabled comments on the core streaming service.  The data science team knows that   this will be an incredibly important source of data going forward.  It will be used inform customer retention, product quality, product market fit and more.  Comments are going live next week and being the diligent data scientist that you are your plan is to build a topic modeling pipeline that that will consume the comments and create visualizations that can be used to communicate with domain experts.
  
## Outline

1. EDA - summary tables, use tSNE to visualize the data
2. Create a transfomation pipelines for NMF and LDA
3. Use ldaviz and wordclouds to get insight into the clusters

## Data

Here we see how to load the data.

* [download the movie review data](http://www.nltk.org/nltk_data)
* For more examples of applications with these data see [NLTK's book chapter that uses these data](https://www.nltk.org/book/ch06.html)

In [2]:
data_dir = os.path.join(".")
movie_reviews = load_files(os.path.join(data_dir,"movie_reviews"), shuffle=True)
X = movie_reviews.data
y = movie_reviews.target
target_names = movie_reviews.target_names
print(X[0])

b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \nwith the help of a trusty sidekick ( kevin pollack ) , they will stop at nothing to let the devil take over the world ! \nparts of this are actually so absurd , that they would fit right in with dogma . \nyes , the film is 

### QUESTION 1

The main focus of this exercise is to enable visualization of topics, but these topics can be used as additional 
features for prediction tasks.  The goal of this case study is to ensure that you are comfortable with natural language processing pipelines and topic modeling tools. 

There are many ways to process tokens (words, dates, emojis etc).  NLTK is often used to pre-process text data before the tokens are vectorized.  Generally, the tokens are modified via [stemming or lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html).  The next code block provides a lemmatization function that makes use of the library [spacy](https://spacy.io/).  You will need to install it and download the English language reference material as follows.  Stopwords are words that are very common or otherwise irrelevant we use a default list here, but it is an important part of NLP pipelines that needs to be customized for the subject area. Use the following function to process the corpus (this can take a few minutes)

```bash
~$ pip install spacy
~$ python -m spacy download en
```

If you prefer to use NLTK then you could use a simple lemmatizer like the WordLemmatizer.

In [3]:
import spacy
STOPLIST = ENGLISH_STOP_WORDS
STOPLIST = set(list(STOPLIST) + ["foo"])

if not 'nlp' in locals():
    print("Loading English Module...")
    nlp = spacy.load('en')

def lemmatize_document(doc, stop_words=None):
    """
    takes a list of strings where each string is a document
    returns a processed list of strings
    """
    
    if not stop_words:
        stop_words = set([])
  
    ## ensure working with string
    doc = str(doc)

    # First remove punctuation form string
    if sys.version_info.major == 3:
        PUNCT_DICT = {ord(punc): None for punc in punctuation}
        doc = doc.translate(PUNCT_DICT)

    # remove unicode
    clean_doc = "".join([char for char in doc if char in printable])
            
    # Run the doc through spaCy
    doc = nlp(clean_doc)

    # Lemmatize and lower text
    tokens = [re.sub("\W+","",token.lemma_.lower()) for token in doc ]
    tokens = [t for t in tokens if len(t) > 1]
    
    return ' '.join(w for w in tokens if w not in stop_words)    

## example usage
corpus = ['"You can fool some of the people all of the time, and all of the people some of the time, but you can not fool all of the people all of the time". -- Abraham Lincoln']
processed = [lemmatize_document(doc, STOPLIST) for doc in corpus]
print(processed[0])
processed = [lemmatize_document(doc, None) for doc in corpus]
print("\n"+processed[0])

Loading English Module...
pron fool people time people time pron fool people time abraham lincoln

pron can fool some of the people all of the time and all of the people some of the time but pron can not fool all of the people all of the time abraham lincoln


In [4]:
processed = [lemmatize_document(doc, STOPLIST) for doc in X]

In [6]:
print(type(processed[0]), type(processed))
processed[0][:1000]

'barnold schwarzenegger icon action enthusiast late 80 lately pron film sloppy oneliner bad nit hard seeing arnold mr freeze batman robin especially pron say ton ice joke hey pron 15 million pron matter pron nonce arnold sign expensive blockbuster compare like terminator series true lie eraser nin dark thriller devil gabriel byrne come earth impregnate woman robin tunney happen 1000 year basically destroy world apparently god choose man man jericho cane arnold pron nwith help trusty sidekick kevin pollack pron stop let devil world npart actually absurd pron fit right dogma nye film weak pron blockbuster right sleepy hollow pron make world look like star film nanyway definitely like arnold movie nit just type film pron pron nsure pron pron chuckle pron know oneliner pron confused pron character film nit understandable especially ending change accord source naside form pron walk pron like pron past film nim sorry say arnold maybe end pron action day nspeake action pron film nthere hardly

### QUESTION 2

Use the CountVectorizer from sklearn to vectorize the tokens.

Additional resources:

* [scikit-learn CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
* [scikit-learn working with text](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

Because this is an exercise in visualization set the `max_features` to something like 1000.  In the context of supervised learning it is reasonable to grid-search to optimize this parameter.

In [8]:
tf_vectorizer = CountVectorizer(max_features = 1000, 
                             min_df = 2, 
                             max_df = 0.95, 
                             stop_words = 'english')
tf = tf_vectorizer.fit_transform(processed)

### QUESTION 3

Use model the corpus with LDA.  For example, you could use something like the following.

```python
n_topics = 10
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

lda_model.fit(tf)
```

You could use a pipeline here to make it easier to iterate on changes.

* [scikit-learn's LDA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)
* [scikit-learn's user guide for LDA](https://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation)

In [9]:
n_components = 10
lda_model = LatentDirichletAllocation(
    n_components = n_components, 
    max_iter = 5, 
    learning_method = 'online', 
    learning_offset = 50., 
    random_state = 0
)
lda_model.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## QUESTION 4

Visualize the corpus using [pyldavis](https://github.com/bmabey/pyLDAvis).

```python
pyLDAvis.sklearn.prepare(lda_model,tf, tf_vectorizer, R=20)
```

* [PyLDAViz documentation](https://pyldavis.readthedocs.io/en/latest)
* [PyLDAViz demos](https://pyldavis.readthedocs.io/en/latest/readme.html#video-demos)

In [10]:
pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, R = 20)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.003008 -0.005134       1        1  38.006099
8     -0.020065 -0.031353       2        1  35.121996
1     -0.005264 -0.034874       3        1  10.571538
5     -0.032837 -0.065788       4        1   6.595433
4     -0.024275 -0.100184       5        1   4.071189
0     -0.009535 -0.047388       6        1   3.930127
2     -0.122270  0.254781       7        1   0.699978
3      0.299756  0.076939       8        1   0.553726
9     -0.068372 -0.028629       9        1   0.395355
6     -0.014129 -0.018372      10        1   0.054558, topic_info=          Term          Freq         Total Category  logprob  loglift
321       film  10959.000000  10959.000000  Default  20.0000  20.0000
603       nthe   7796.000000   7796.000000  Default  19.0000  19.0000
838       star   1209.000000   1209.000000  Default  18.0000  18.0000
537      movie   6628.000000   6628.000000  Default  17.0000  17.0000
10      action   1146.000000   1146.000000  Default  16.0000  16.0000
..         ...           ...           ...      ...      ...      ...
484       look      0.330704   1727.428810  Topic10  -6.3980  -1.0473
137  character      0.362509   3809.801287  Topic10  -6.3062  -1.7464
606     nthere      0.306086   1231.813527  Topic10  -6.4754  -0.7865
493       make      0.328343   4073.737712  Topic10  -6.4052  -1.9124
495        man      0.308548   1827.670376  Topic10  -6.4674  -1.1730

[394 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
7         1  0.123301  accident
7         2  0.369902  accident
7         3  0.016440  accident
7         4  0.032880  accident
7         5  0.402783  accident
...     ...       ...       ...
996       5  0.034876      year
996       6  0.025189      year
996       7  0.000646      year
996       8  0.002583      year
996       9  0.001292      year

[1847 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 2, 6, 5, 1, 3, 4, 10, 7])

In [11]:
pylda = pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, R = 20)
pyLDAvis.show(pylda)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [13/Jun/2020 06:43:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:43:06] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:43:06] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:43:06] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:43:07] code 404, message Not Found
127.0.0.1 - - [13/Jun/2020 06:43:07] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Jun/2020 06:45:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:45:49] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:45:49] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:45:49] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


## QUESTION 5

Try different numbers of clusters until there is decent separation in the visualization

In [24]:
n_components = 4
lda_model = LatentDirichletAllocation(n_components = n_components, max_iter = 5, learning_method = 'online', learning_offset = 50., random_state = 0)
lda_model.fit(tf)
pyLDAvis.show(pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, R = 20))


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [13/Jun/2020 06:56:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:56:39] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:56:39] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2020 06:56:39] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


The visualization here can help determine a reasonable number of number of clusters and it can serve as a communication tool.  If the goal was to find topics that are associated with customer profiles then you would likely work with folks in marketing to refine the clustering.  There are a couple of parameters than can be used to modify the clustering and visualization.  The discovery of meaningful topics is a form of feature engineering.

## QUESTION 6

If you were to use the topics from this model to inform clustering or supervised learning you would first need to be able to extract and represent them as a matrix.  Along the same lines if you were to populate a report with tabular descriptions of the data then you will need to be able to extract topic representations.  Here is a starter function

```python
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return(top_words)
```

Use the function to print the top k words for each topic

In [28]:
tf_vectorizer.get_feature_names()

['10',
 '90',
 'ability',
 'able',
 'absolutely',
 'accent',
 'accept',
 'accident',
 'act',
 'acting',
 'action',
 'actor',
 'actress',
 'actual',
 'actually',
 'add',
 'admit',
 'adult',
 'adventure',
 'affair',
 'age',
 'agent',
 'ago',
 'agree',
 'air',
 'alien',
 'alive',
 'allen',
 'allow',
 'amazing',
 'america',
 'american',
 'amusing',
 'angel',
 'animal',
 'animate',
 'animation',
 'annoying',
 'answer',
 'apartment',
 'ape',
 'apparently',
 'appeal',
 'appear',
 'appearance',
 'appreciate',
 'approach',
 'army',
 'arrive',
 'art',
 'artist',
 'ask',
 'aspect',
 'atmosphere',
 'attack',
 'attempt',
 'attention',
 'audience',
 'avoid',
 'award',
 'away',
 'awful',
 'baby',
 'background',
 'bad',
 'band',
 'base',
 'basic',
 'basically',
 'batman',
 'battle',
 'bear',
 'beat',
 'beautiful',
 'beauty',
 'begin',
 'beginning',
 'believable',
 'believe',
 'ben',
 'best',
 'better',
 'big',
 'bit',
 'bizarre',
 'black',
 'blood',
 'blow',
 'blue',
 'bob',
 'body',
 'bond',
 'book',

In [34]:
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return top_words

features = get_top_words(lda_model, tf_vectorizer.get_feature_names(), 20)
features.items()

dict_items([('0', ['film', 'movie', 'nthe', 'like', 'make', 'good', 'just', 'character', 'nit', 'bad', 'ni', 'time', 'scene', 'play', 'know', 'really', 'nand', 'thing', 'think', 'come']), ('1', ['film', 'nthe', 'character', 'make', 'story', 'nit', 'like', 'life', 'good', 'time', 'man', 'movie', 'scene', 'nhe', 'love', 'way', 'play', 'nin', 'work', 'come']), ('2', ['film', 'movie', 'nthe', 'make', 'character', 'time', 'scene', 'like', 'horror', 'play', 'scream', 'good', 'killer', 'know', 'plot', 'just', 'vampire', 'jackie', 'nit', 'director'])])

## QUESTION (EXTRA CREDIT) 7

If you used `fit_transform` on your original tokens you should have a `2000 x k` array where `k` is the number of topics you choose.  Create a PCA or tSNE visualization that projects the tf matrix into lower dimensional space then uses colors to indicate which documents belong to a topic (e.g. probability > 0.5).

<iframe src="https://player.vimeo.com/video/87110435" width="640" height="360"  frameborder="0" allowfullscreen></iframe>